In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
root=os.path.join('/content','drive','My Drive','Colab Notebooks','kaggle_Quora')

In [3]:
import pandas as pd
train=pd.read_csv(os.path.join(root,'train.csv'))
test1=pd.read_csv(os.path.join(root,'test.csv'))
test2=pd.read_csv(os.path.join(root,'test (1).csv'))


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
q1=list(train['question1'])
q2=list(train['question2'])

test_q1=list(test1['question1'])
test_q2=list(test1['question2'])

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
import re
import string
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

q1_without_stop=[]
table = str.maketrans('', '', string.punctuation)

i=0
q1=q1[:105779]
for qw in q1:
  ##print(qw)
  words=qw.split()
  
  stripped=[w.translate(table) for w in words]
  new_line=[]
  
  for word in stripped:
    if word not in stop:
      word=word.lower()
      new_line.append(word)

  q1_without_stop.append(new_line)
  
print(q1_without_stop[0:5])

[['what', 'step', 'step', 'guide', 'invest', 'share', 'market', 'india'], ['what', 'story', 'kohinoor', 'kohinoor', 'diamond'], ['how', 'i', 'increase', 'speed', 'internet', 'connection', 'using', 'vpn'], ['why', 'i', 'mentally', 'lonely', 'how', 'i', 'solve'], ['which', 'one', 'dissolve', 'water', 'quikly', 'sugar', 'salt', 'methane', 'carbon', 'di', 'oxide']]


In [7]:
import re
import string
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

q2_without_stop=[]
table = str.maketrans('', '', string.punctuation)
q2=q2[:105779]
i=0

for qw in q2:
  ##print(qw)
  words=qw.split()
  
  stripped=[w.translate(table) for w in words]
  new_line=[]
  
  for word in stripped:
    if word not in stop:
      word=word.lower()
      new_line.append(word)
  
  q2_without_stop.append(new_line)

print(q2_without_stop[0:5])

[['what', 'step', 'step', 'guide', 'invest', 'share', 'market'], ['what', 'would', 'happen', 'indian', 'government', 'stole', 'kohinoor', 'kohinoor', 'diamond', 'back'], ['how', 'internet', 'speed', 'increased', 'hacking', 'dns'], ['find', 'remainder', 'math2324math', 'divided', '2423'], ['which', 'fish', 'would', 'survive', 'salt', 'water']]


In [8]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

MAX_NUM_WORDS=5000
MAX_SEQUENCE_LENGTH=200

tokenizer=Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(q1_without_stop)
word_index=tokenizer.word_index

seq=tokenizer.texts_to_sequences(q1_without_stop)
print(f'Number of uniques words {len(word_index)}')


data=pad_sequences(seq,maxlen=MAX_SEQUENCE_LENGTH)

Using TensorFlow backend.


Number of uniques words 42480


In [0]:
import numpy as np

In [10]:
embeddings_index = {}
f = open('/content/drive/My Drive/Colab Notebooks/Glove/glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')

GloVe data loaded


In [11]:
print(embeddings_index.get('a').shape[0])

300


In [12]:
from keras.layers import Embedding

from keras.initializers import Constant

EMBEDDING_DIM=embeddings_index.get('a').shape[0]
num_words=min(MAX_NUM_WORDS,len(word_index))+1
embedding_matrix=np.zeros((num_words,EMBEDDING_DIM))

for word, i in word_index.items():
  if i>MAX_NUM_WORDS:
    continue
  embedding_vector=embeddings_index.get(word)

  if embedding_vector is not None:
    embedding_matrix[i]=embedding_vector

  
embedding_layer=Embedding(num_words, EMBEDDING_DIM,embeddings_initializer=Constant(embedding_matrix),input_length=MAX_SEQUENCE_LENGTH,trainable=False)


In [0]:
embeddings_index.get('apple')

In [0]:
embedded_q1=[]
for sent in q1_without_stop:
  emb_sen=[]
  for word in sent:
    emb_sen.append(embeddings_index.get('word'))

  embedded_q1.append(emb_sen)
  

In [0]:
embedded_q2=[]
for sent in q2_without_stop:
  emb_sen=[]
  for word in sent:
    emb_sen.append(embeddings_index.get('word'))

  embedded_q2.append(emb_sen)
  

In [0]:
len(embedded_q1),len(embedded_q2[0])

qs=np.array(embedded_q1[0])

In [0]:
dif=[]
for i in range(105779):
  q1_1=np.array(embedded_q1[i])
  q2_1=np.array(embedded_q2[i])
  
  word_len_max=max(q1_1.shape[0],q2_1.shape[0])
  q1_2=np.zeros((word_len_max,300))
  q2_2=np.zeros((word_len_max,300))
  
  q1_2[:q1_1.shape[0],:q1_1.shape[1]] = q1_1

  q2_2[:q2_1.shape[0],:q2_1.shape[1]] = q2_1
  
  dif.append(np.mean(q1_2-q2_2))

In [18]:
len(dif)

105779

In [19]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
train_diff=pd.concat((pd.Series(dif),train['is_duplicate'][:105779]),axis=1)

In [21]:
train_diff

,0,is_duplicate
0,-0.001004,0
1,0.004016,0
2,-0.002008,0
3,-0.002295,0
4,-0.003651,0
...,...,...
105774,-0.001606,1
105775,0.004016,1
105776,0.000000,1
105777,0.001606,1


In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
from sklearn.model_selection import train_test_split
X=pd.DataFrame(train_diff[0])
y=train_diff['is_duplicate']
train_x, test_x, train_y, test_y = train_test_split(X,
y, train_size=0.8, random_state = 5)

In [24]:
X.shape

(105779, 1)

In [25]:
lr=LogisticRegression()
lr.fit(train_x,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score

print("Multinomial Logistic regression Train Accuracy : ",accuracy_score(train_y, lr.predict(train_x)))
print("Multinomial Logistic regression Test Accuracy : ", accuracy_score(test_y, lr.predict(test_x)))

Multinomial Logistic regression Train Accuracy :  0.6295924275906077
Multinomial Logistic regression Test Accuracy :  0.6211476649650217


In [27]:
from sklearn.metrics import log_loss
log_loss(test_y,lr.predict_proba(test_x)[:,1])

0.6636519257953751

In [28]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_leaf_nodes=50, n_estimators=50, min_samples_leaf=10)
rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=50, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
log_loss(test_y,rf.predict_proba(test_x)[:,1])

0.6338080534936003

In [0]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(train_x, train_y)
print("SVM Train Accuracy : ",accuracy_score(train_y, svc.predict(train_x)))
print("SVM Test Accuracy : ", accuracy_score(test_y, svc.predict(test_x)))